In [68]:
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt


In [69]:
np.random.seed(42)

def load_and_preprocess_data():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    X_train = X_train.reshape(-1, 28*28)
    X_test = X_test.reshape(-1, 28*28)
    y_train_binary = (y_train == 0).astype(int)
    y_test_binary = (y_test == 0).astype(int)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, y_train_binary, X_test, y_test_binary

In [70]:
def reduce_dimensionality(X_train, X_test, n_components=10):
    pca = PCA(n_components=n_components)
    X_train_reduced = pca.fit_transform(X_train)
    X_test_reduced = pca.transform(X_test)
    print(f"PCA Explained Variance: {np.sum(pca.explained_variance_ratio_):.4f}")
    return X_train_reduced, X_test_reduced

In [71]:

# ANFIS Implementation
class ANFIS:
    def __init__(self, n_inputs, n_rules):
        self.n_inputs = n_inputs
        self.n_rules = n_rules
        self.params = self.initialize_parameters()

    def initialize_parameters(self):
        params = {
            'mf_params': np.zeros((self.n_rules, self.n_inputs, 2)),
            'consequent_params': np.random.randn(self.n_rules, 1) * 0.1
        }
        params['mf_params'][:, :, 0] = np.random.randn(self.n_rules, self.n_inputs) * 0.1
        params['mf_params'][:, :, 1] = np.abs(np.random.randn(self.n_rules, self.n_inputs)) + 0.5
        return params
    
    def gaussian_mf(self, x, params):
        center, sigma = params[:, :, 0], np.abs(params[:, :, 1]) + 1e-8
        memberships = np.exp(-((x[:, None, :] - center) ** 2) / (2 * sigma ** 2))
        memberships = np.clip(memberships, 1e-8, 1.0)
        return memberships
    
    def forward(self, X):
        memberships = self.gaussian_mf(X, self.params['mf_params'])
        firing_strengths = np.prod(memberships, axis=2)
        firing_sum = np.sum(firing_strengths, axis=1, keepdims=True)
        normalized_strengths = firing_strengths / (firing_sum + 1e-8)
        consequent = np.dot(normalized_strengths, self.params['consequent_params'])
        return consequent.squeeze()
    
    def train(self, X, y, epochs=50, lr=0.05):
        for epoch in range(epochs):
            output = self.forward(X)
            loss = np.mean((output - y) ** 2)
            grad_output = 2 * (output - y) / X.shape[0]
            memberships = self.gaussian_mf(X, self.params['mf_params'])
            firing_strengths = np.prod(memberships, axis=2)
            firing_sum = np.sum(firing_strengths, axis=1, keepdims=True)
            normalized_strengths = firing_strengths / (firing_sum + 1e-8)
            grad_consequent = np.dot(normalized_strengths.T, grad_output[:, None])
            self.params['consequent_params'] -= lr * grad_consequent
            error = output - y
            grad_mf_center = np.zeros_like(self.params['mf_params'][:, :, 0])
            grad_mf_sigma = np.zeros_like(self.params['mf_params'][:, :, 1])
            for r in range(self.n_rules):
                for i in range(self.n_inputs):
                    diff = X[:, i] - self.params['mf_params'][r, i, 0]
                    sigma = np.abs(self.params['mf_params'][r, i, 1]) + 1e-8
                    grad_mf_center[r, i] = np.mean(error * normalized_strengths[:, r] * diff / (sigma ** 2))
                    grad_mf_sigma[r, i] = np.mean(error * normalized_strengths[:, r] * (diff ** 2) / (sigma ** 3))
            self.params['mf_params'][:, :, 0] -= lr * grad_mf_center * 0.1
            self.params['mf_params'][:, :, 1] -= lr * grad_mf_sigma * 0.1
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {accuracy_score(y, np.round(output)):.4f}")

In [72]:
X_train, y_train, X_test, y_test = load_and_preprocess_data()
X_train_reduced, X_test_reduced = reduce_dimensionality(X_train, X_test, n_components=10)
anfis_model = ANFIS(n_inputs=10, n_rules=5)
anfis_model.train(X_train_reduced, y_train, epochs=100, lr=0.1)
y_pred = np.round(anfis_model.forward(X_test_reduced))
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy (Binary - '0' vs not '0'): {test_accuracy:.4f}")
plt.figure(figsize=(8, 6))
plt.scatter(X_test_reduced[:, 0], X_test_reduced[:, 1], c=y_pred, cmap='coolwarm', alpha=0.6)
plt.title("ANFIS Predicted Labels (PCA Reduced to 2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar(label='Predicted Label')
plt.savefig("anfis_mnist_binary_classification.png")
plt.close()


PCA Explained Variance: 0.2767
Epoch 0, Loss: 0.0986, Accuracy: 0.9013
Epoch 10, Loss: 0.0985, Accuracy: 0.9013
Epoch 20, Loss: 0.0985, Accuracy: 0.9013
Epoch 30, Loss: 0.0985, Accuracy: 0.9013
Epoch 40, Loss: 0.0985, Accuracy: 0.9013
Epoch 50, Loss: 0.0985, Accuracy: 0.9013
Epoch 60, Loss: 0.0985, Accuracy: 0.9013
Epoch 70, Loss: 0.0985, Accuracy: 0.9013
Epoch 80, Loss: 0.0985, Accuracy: 0.9013
Epoch 90, Loss: 0.0985, Accuracy: 0.9013
Test Accuracy (Binary - '0' vs not '0'): 0.9020
